In [1]:
import numpy as np, pandas as pd
from scipy.sparse import coo_matrix
from poismf import PoisMF
from pandas.api.types import CategoricalDtype
from scipy import sparse
from sklearn.model_selection import train_test_split
import tqdm


In [20]:
train_df = pd.read_csv("pf_last_netflix_time.tsv",sep="\t",    names=[
        'user_id', 'movie_id', 'rating', 'timestamp'
    ],)
test_df = pd.read_csv("pf_last_netflix_time_test.tsv",sep="\t",    names=[
        'user_id', 'movie_id', 'rating', 'timestamp'
    ],)


In [22]:
test_users = test_df['user_id'].unique()
test_items = test_df['movie_id'].unique()


train_users = train_df['user_id'].unique()
train_items = train_df['movie_id'].unique()


user = list(set(train_users) & set(test_users))
item = list(set(train_items) & set(test_items))


In [ ]:
train_df = train_df[train_df['user_id'].isin(user)]
train_df = train_df[train_df['movie_id'].isin(item)]

test_df = test_df[test_df['user_id'].isin(user)]
test_df = test_df[test_df['movie_id'].isin(item)]

In [24]:

test_ratings_matrix = test_df.pivot(index='user_id', columns='movie_id', values='rating')
train_ratings_matrix = train_df.pivot(index='user_id', columns='movie_id', values='rating')


In [25]:
display(train_ratings_matrix)
display(test_ratings_matrix)

movie_id,16,18,44,46,47,55,77,79,81,83,...,17681,17692,17709,17725,17726,17730,17732,17756,17762,17764
user_id,,,,,,,,,,,,,,,,,,,,,
2976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2643933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2645828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2646634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


movie_id,16,18,46,47,55,77,79,81,83,96,...,17692,17709,17725,17726,17730,17732,17746,17756,17762,17764
user_id,,,,,,,,,,,,,,,,,,,,,
2976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2643933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2645828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2646634,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
user_positives = test_df.groupby('user_id')['movie_id'].apply(set).to_dict()


In [8]:
train_df =train_df.drop("timestamp",axis=1)
train_df.columns =["UserId", "ItemId", "Count"]

display(train_df)


,UserId,ItemId,Count
1,2454575,16,1
2,1075509,16,1
3,170248,16,1
4,1814763,18,1
5,61765,18,1
...,...,...,...
39144,1225643,17764,1
39145,2625242,17764,1
39148,185339,17764,1
39151,617221,17764,1


In [9]:
modelPF = PoisMF(reindex=True, method="tncg", use_float=True,
                    early_stop=False, reuse_prev=True,
                    k=20, niter=10, maxupd=200, l2_reg=1e3)\
                .fit(train_df)

In [10]:
display(modelPF.A.shape)
display(modelPF.B.shape)


(1693, 20)

(2723, 20)

In [ ]:
pred = np.matmul(modelPF.A,modelPF.B.T)

In [ ]:
from recsys_metrics import *
import torch
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation

In [ ]:
display(pred.shape)

(6163, 4105)

In [ ]:
test_ratings_matrix[test_ratings_matrix >1] = 1

true = torch.tensor(test_ratings_matrix.fillna(0).to_numpy())


In [ ]:
predv = torch.tensor(pred)


In [ ]:
display(true)
display(predv)

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

tensor([[0.2040, 0.6408, 0.2832,  ..., 0.0087, 0.0134, 0.0108],
        [0.2045, 0.6424, 0.2839,  ..., 0.0087, 0.0135, 0.0108],
        [0.2048, 0.6434, 0.2843,  ..., 0.0087, 0.0135, 0.0108],
        ...,
        [0.0319, 0.1001, 0.0443,  ..., 0.0013, 0.0021, 0.0017],
        [0.0492, 0.1546, 0.0683,  ..., 0.0021, 0.0032, 0.0026],
        [0.0298, 0.0936, 0.0414,  ..., 0.0013, 0.0020, 0.0016]])

In [ ]:
def calculate_metrics(pred_df, user_postivies, k_list = [10,50,100]):
  metrics = {
    'MAR@ALL': [],
    'MAR@1000': [],
  }


  grouped = pred_df.groupby('user')

  for user, user_data in tqdm.tqdm(grouped, desc="Processing Users"):
    positives = user_postivies.get(user, set())
    if len(positives) == 0:
      continue

    sorted_data = user_data.sort_values(by='score', ascending=False)
    all_items = sorted_data['item'].tolist()

    relevant_ranks1k = []
    relevant_ranks_all=[]
    for rank, item in enumerate(all_items[:1000], start=1):
      if item in positives:
        relevant_ranks1k.append(rank)
    #calculate relevant
    for rank, item in enumerate(all_items, start=1):
      if item in positives:
        relevant_ranks_all.append(rank)



    if relevant_ranks_all:
      metrics['MAR@ALL'].append(np.mean(relevant_ranks_all))
    if relevant_ranks1k:
      metrics['MAR@1000'].append(np.mean(relevant_ranks1k))
      

  
  final = {
    'MAR@ALL': np.mean(metrics['MAR@ALL']),
    'MAR@1000': np.mean(metrics['MAR@1000']),
  }

  return final

In [ ]:
def metricStuff(pred,true,user_positive):

    print("NDCG@10",normalized_dcg(pred, true ,k=10))
    print("MRR@1000",mean_reciprocal_rank(pred, true,k=1000))
    print("MRR@all",mean_reciprocal_rank(pred, true))
    marData = pred.detach().numpy()
    marDF = pd.DataFrame(marData)
    marDF.index.name = "user"
    marDF.columns.name = "items"
    marDF = marDF.stack().reset_index()
    marDF.columns = ['user', 'item', 'score']
    print("MAR",calculate_metrics(marDF,user_positive))
    print("MAP",mean_average_precision(pred, true))
    print("P@50",precision(pred, true,k=50))
    print("R@50",recall(pred, true,k=50))
    

In [ ]:
metricStuff(predv,true,user_positives)

NDCG@10 tensor(0.0568)
MRR@1000 tensor(0.1421)
MRR@all tensor(0.1421)


Processing Users: 100%|██████████| 6163/6163 [00:01<00:00, 4947.77it/s]


MAR {'MAR@ALL': np.float64(2141.644424031392), 'MAR@1000': np.float64(538.5314443132625)}
MAP tensor(0.0423)
P@50 tensor(0.0453)
R@50 tensor(0.0140)
